In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing, model_selection, neighbors, svm, discriminant_analysis
from sklearn.feature_selection import SelectPercentile,SelectFpr,SelectKBest, chi2
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
train = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [3]:
df = pd.DataFrame(test["PassengerId"], index=None)
df

,PassengerId
0,0013_01
1,0018_01
2,0019_01
3,0021_01
4,0023_01
...,...
4272,9266_02
4273,9269_01
4274,9271_01
4275,9273_01


In [4]:
# Split cabin by deck, num and side
def CabinSplit(df):
    df["Deck"] = df["Cabin"].str.split("/").str[0]
    df["Num"] = list(map(float, df["Cabin"].str.split("/").str[1]))
    df["Side"] = df["Cabin"].str.split("/").str[2]
    df = df.drop(["Cabin", "PassengerId", "Destination", "Name"], axis=1)
    return df
#Fill Missing values
def FillNaN(df):
    mf = ["HomePlanet", "CryoSleep", "VIP", "Side", "Deck"]
    mean = ["Age"]
    zero = ["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck", "Num"]
    df[mf] = df[mf].fillna(df.mode().iloc[0])
    df[mean] = df[mean].fillna(df[mean].mean())
    df[zero] = df[zero].fillna(0)
    return df
#Feature engineering
def FeatureCreation(df):
    df["MoneySpent"] = df["RoomService"] + df["FoodCourt"] + df["ShoppingMall"] + df["Spa"] + df["VRDeck"]
    df.insert(loc = 3, column="AgeCategories",value=0)
    df.loc[df["Age"] <= 14, "AgeCategories"] = 1
    df.loc[(df["Age"] > 14) & (df["Age"] <= 24), "AgeCategories"] = 2
    df.loc[(df["Age"] > 24) & (df["Age"] <= 64), "AgeCategories"] = 3
    df.loc[(df["Age"] > 64), "AgeCategories"] = 4
    return df
#Encode strings  to numericals
def Encode(df):
    df = pd.concat([df, pd.get_dummies(df[["Deck", "Side", "HomePlanet"]])], axis=1)
    df = pd.concat([df, pd.get_dummies(df["AgeCategories"])], axis=1)
    df[["CryoSleep", "VIP"]] = df[["CryoSleep", "VIP"]].apply(LabelEncoder().fit_transform)
    df = df.drop(["Deck", "Side", "HomePlanet", "AgeCategories"], axis=1)
    return df
#Scale everything
def Norm(df):
    for col in df:
        if df[col].dtypes == "float64":
            df[col] = StandardScaler().fit_transform(np.array(df[col]).reshape(-1, 1))
    return df
def preprocess(df):
    df = CabinSplit(df)
    df = FillNaN(df)
    df = FeatureCreation(df)
    df = Encode(df)
    df = Norm(df)
    return df

In [5]:
target = train["Transported"]
train.drop(columns="Transported", inplace=True)

train = preprocess(train)
test = preprocess(test)


In [6]:

accr=[]
X_train, X_test, y_train, y_test = model_selection.train_test_split(train, target, test_size=0.2, random_state=11)
#Classifiers (uncomment one to run with that model) SVM can be entered here, later
#clf = discriminant_analysis.LinearDiscriminantAnalysis(tol=0.05)
#clf =  neighbors.KNeighborsClassifier(n_neighbors=23,n_jobs=-1)
#clf = discriminant_analysis.QuadraticDiscriminantAnalysis(tol=0.05)
clf = svm.SVC(kernel='rbf', random_state=1)

'''
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=7)
'''


# Model fit on training set
clf.fit(X_train,y_train)

#clf.fit(X_train, y_train,epochs=50 )
#
#Accuracy found on testing set or hold on set, uncomment either before running
accuracy = clf.score(X_test, y_test)
#accuracy = clf.score(X_unk, y_unk)
#print(accuracy)
#accuracy=clf.score(X_unk,y_unk)

# Accuracy may also be calculated by comparing labels to a predicted output of the model as done here
#preds=clf.predict(X_unk)
#accuracy = accuracy_score(y_unk, preds)
#Accuracy appended in list
accr.append(accuracy)
print(sum(accr)/len(accr))

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


0.8050603795284647


In [7]:
df["Transported"]=clf.predict(test)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [9]:
df.to_csv('submission.csv', index=False)